In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
!ls
os.chdir("drive/MyDrive/Snake Images")
!ls


Mounted at /content/drive
drive  sample_data
models	test  train


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten
from tensorflow.keras.regularizers import l2

In [ ]:
train_dir = "dataset/training_set/"

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
image_df = image_df.sample(frac=1).reset_index(drop = True)

# Separate in train and test data
train_df, test_df = train_test_split(image_df, train_size=0.9, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = (1/255.), shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = (1/255.))

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training')

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation')
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False)

Found 1688 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 2 classes.
Found 188 validated image filenames belonging to 2 classes.


In [ ]:
def create_model(input_shape=(224, 224, 3)):
    
    inputs = Input(input_shape)
    base_model = Sequential()
    
    x = base_model(inputs)
    
    x = Conv2D(filters = 32, padding = "same", activation = "relu", kernel_size = 3, strides = 2, input_shape = (224,224,3))(x)
    x = MaxPool2D(pool_size = (2,2), strides = 2)(x)
    
    x = Conv2D(filters = 32, padding = "same", activation = "relu", kernel_size = 3)(x)
    x = MaxPool2D(pool_size = (2,2), strides = 2)(x)

    x = Flatten()(x)
    x = Dense(128,activation = "relu")(x)
    
    outputs = Dense(1,kernel_regularizer = l2(0.01), activation = "linear")(x)
    
    model = Model(inputs, outputs)
    
    return model

In [ ]:
K.clear_session()

model = create_model((224, 224, 3))
# model = load_model('models/checkpoint/EfficientNetB0.h5')

metrics = [
    'accuracy',
    'AUC'
]

In [ ]:
model.compile(optimizer = Adam(), loss = tf.keras.losses.BinaryCrossentropy(), metrics=metrics)

In [ ]:
history = model.fit(
    x = train_images,
    validation_data=val_images,
    epochs=25)

Epoch 1/25


InvalidArgumentError: ignored